In [183]:
import math
import time

import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from torchvision.transforms import Lambda
from torch.nn import functional as F
from scipy import io as sio

In [184]:
# set the selected device for the tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(torch.cuda.get_device_name(device))
torch.set_default_device(device)

Using device: cuda
NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4330, 0.3819, 0.2964],std=[0.2588, 0.2094, 0.2212])
])

In [ ]:
test_dataset = datasets.Flowers102(
    root='flowers102',
    split="test",
    download=True,
    transform=test_transform,
)

In [ ]:
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size, shuffle=True, num_workers=2)

In [ ]:
class MyNN4(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = self.make_conv(3, 32, 3, 1, 1)
        self.conv2 = self.make_conv(32, 32, 3, 1, 1)  
        
        self.conv3 = self.make_conv(32, 64, 3, 1, 1)
        self.conv4 = self.make_conv(64, 64, 3, 1, 1)  
        
        self.conv5 = self.make_conv(64, 128, 3, 1, 1)
        self.conv6 = self.make_conv(128, 128, 3, 1, 1)
        self.conv7 = self.make_conv(128, 128, 3, 1, 1)
        
        self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.flatten = nn.Flatten()
        
        self.fc1 = self.fully_connected(128*28*28, 4096)
        self.fc2 = self.fully_connected(4096, 1024)
        
        self.fc3 = nn.Linear(1024, 102)
        
    def make_conv(self, in_feat, out_feat, kernel, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels=in_feat, out_channels=out_feat, kernel_size=kernel, stride=stride, padding=padding),
            nn.BatchNorm2d(out_feat),
            nn.ReLU()
        )
    def fully_connected(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Linear(in_features=in_channels, out_features=out_channels),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        x = self.maxpool(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
model = MyNN4().to(device)
model.load_state_dict(torch.load("./checkpoints/checkpoint.pth"))

In [ ]:
def test(model, valid_loader):
    model.eval()
    with torch.no_grad():
        acc = .0
        for i, data in enumerate(valid_loader):
            X = data[0].to(device)
            y = data[1].to(device)

            predicted = model(X)
            
            # Check each image's prediction
            for id in range(len(X)):
                if torch.argmax(predicted[id]).detach().item() == torch.argmax(y[id]).detach().item():
                    acc += 1
    model.train()
    return acc/len(valid_loader.dataset)